# DrugEx API

An example DrugEx workflow showcasing some basic DrugEx API features. The API provides interface definitions to handle data operations and training of models needed for obtaining a molecule designer. The interface should ensure that the current code base is extensible and loosely coupled to make interoperability with different data sources seamless and to also aid in monitoring of the training processes involved.

Let's import and explain some of the important API features:

In [1]:
# main package
import drugex

# important classes for data access
from drugex.api.environ.data import ChEMBLCSV
from drugex.api.corpus import CorpusCSV, BasicCorpus, CorpusChEMBL

# important classes for QSAR modelling
# and (de)serialization of QSAR models
from drugex.api.environ.models import RF
from drugex.api.environ.serialization import FileEnvSerializer, FileEnvDeserializer

# classes that handle training of the exploration
# and exploitation networks and also handle monitoring 
# of the process 
from drugex.api.model.callbacks import BasicMonitor
from drugex.api.pretrain.generators import BasicGenerator

# ingredients needed for DrugEx agent training
from drugex.api.agent.agents import DrugExAgent
from drugex.api.agent.callbacks import BasicAgentMonitor
from drugex.api.agent.policy import PG

# designer API (wraps the agent after it was trained)
from drugex.api.designer.designers import BasicDesigner, CSVConsumer

Next let's define some global settings:

In [2]:
import torch

for device in range(torch.cuda.device_count()):
    print(device, torch.cuda.get_device_capability(device))

0 (3, 5)
1 (2, 1)
2 (3, 5)


/home/sichom/software/miniconda/envs/drugex/lib/python3.5/site-packages/torch/cuda/__init__.py:117: UserWarning: 
    Found GPU1 GeForce GT 610 which is of cuda capability 2.1.
    PyTorch no longer supports this GPU because it is too old.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [3]:
import os

if torch.cuda.is_available():
    # choose a GPU device based on the info above
    # (the higher the capability, the better)
    torch.cuda.set_device(2)

DATA_DIR="data" # folder with input data files
OUT_DIR="output/workflow" # folder to store the output of this workflow
os.makedirs(OUT_DIR, exist_ok=True) # create the output folder

# define a set of gene IDs that are interesting for 
# the target that we want to design molecules for
GENE_IDS = ["ADORA2A"]

## Data Aquisition

It's time to aquire the data we will need for training of our models. There are three models that we need to build so we need three separate data sets:

1. Data for the exploitation model based on a random sample of 1 million molecules from the ZINC set.
2. Data for the exploration model based on ChEMBL data we downloaded for the desired target.
3. Data for the QSAR modelling of the environment model -> this model will bias the final generator towards more active molecules throug the a policy gradient.

### Exploitation Network

The exploitation network will be based on a large data set of known chemical structures. The ZINC database is a great source of data for the network:

In [ ]:
# Randomly selected sample of 1 million molecules
# from the ZINC database.
# We only use this file for illustration purposes.
# In practice, the pretrained exploitation network should
# be provided so there will be no need for this data,
# but we are starting from square one here.
ZINC_CSV=os.path.join(DATA_DIR, "ZINC.txt")

# Load SMILES data into a corpus from a CSV file (we assume
# that we have the structures saved in a csv file in DATA_DIR).
# Corpus is a class which provides both the vocabulary and
# training data for a generator.
# This corupus will be used to train the exploitation network later.
corpus_pre = CorpusCSV(
    update_file=ZINC_CSV
    # The input CSV file with chemical structures as SMILES.
    # This is the only required parameter of this class.

    , vocabulary=drugex.VOC_DEFAULT
    # A vocabulary object that defines the tokens
    # and other options used to construct and parse SMILES.
    # VOC_DEFAULT is a reasonable "catch all" default.

    , smiles_column="CANONICAL_SMILES"
    # Instructs the corpus object what column to look for when
    # extracting SMILES to update the data.

    , sep='\t'
    # The column separator used in the CSV file
)

# Next we update the corpus (if we did not do it already).
# The updateData() method loads and tokenizes the SMILES it finds in the CSV.
# The tokenized data and updated vocabulary are returned to us.
corpus_out_zinc = os.path.join(OUT_DIR, "zinc_corpus.txt")
vocab_out_zinc = os.path.join(OUT_DIR, "zinc_voc.txt")
if not os.path.exists(corpus_out_zinc):
    df, voc = corpus_pre.updateData(update_voc=True)
    # We don't really use the return values here, but they are
    # still there if we need them for logging purposes or
    # something else. The update_voc flag tells the 
    # update method to also update the vocabulary
    # based on the tokens found in the SMILES strings.

    # We can save our corpus data if we want to reuse it later.
    # The CorpusCSV class has a methods
    # that we can use to save the vocabulary and tokenized data set.
    corpus_pre.saveCorpus(corpus_out_zinc)
    corpus_pre.saveVoc(vocab_out_zinc)
else:
    # If we initialized and saved
    # the corpus before, we just overwrite the
    # current one with the saved one
    corpus_pre= CorpusCSV.fromFiles(corpus_out_zinc, vocab_out_zinc)

Reading SMILES: 100%|██████████| 1018452/1018452 [00:03<00:00, 270090.17it/s]

### Exploration Network

We will also need a corpus for the exploration network. We will load it from ChEMBL using a different implementation of the Corpus interface than we saw above. When we update a CorpusChEMBL instance, it downloads the data for us automatically:

In [ ]:
# CorpusChEMBL uses a list of gene identifiers
# and download activity data for all tested compounds
# related to the particular genes.
corpus_out_chembl = os.path.join(OUT_DIR, "chembl_corpus.txt")
vocab_out_chembl = os.path.join(OUT_DIR, "chembl_voc.txt")
env_data_path = os.path.join(OUT_DIR, "{0}.txt".format(GENE_IDS[0]))
if not os.path.exists(corpus_out_chembl):
    corpus_ex = CorpusChEMBL(GENE_IDS, clean_raw=True)

    # lets update this corpus and save the results
    # (same procedure as above)
    df, voc = corpus_ex.updateData(update_voc=True)
    corpus_ex.saveCorpus(corpus_out_chembl)
    corpus_ex.saveVoc(vocab_out_chembl)

    # in addition we will also save the raw downloaded data
    # (this is what we will also use as a basis for the environment QSAR model)
    corpus_ex.raw_data.to_csv(env_data_path, sep="\t", index=False)
else:
    # If we already generated the corpus file,
    # we can load it using the CorpusCSV class
    corpus_ex = CorpusCSV.fromFiles(corpus_out_chembl, vocab_out_chembl)

Since in both cases we requested to update the vocabulary according to
tokens found in the underlying smiles for both the zinc
and ChEMBL corpus, we now need to unify them. Vocabularies
can be combined using the plus operator:

In [ ]:
voc_all = corpus_pre.voc + corpus_ex.voc
corpus_pre.voc = voc_all
corpus_ex.voc = voc_all

If we did not do this, the exploitation and
exploration networks might not be compatible
and we would run into issues during modelling.

### Environment QSAR model

We also need activity data to
train the environment QSAR model which will provide the activity
values for policy gradient.
Luckily, we already have the file to do this:

In [ ]:
environ_data = ChEMBLCSV(
    env_data_path # we got this file from ChEMBL thanks to CorpusChEMBL
    , 6.5 # this is the activity threshold for the pChEMBL value
    , id_col='MOLECULE_CHEMBL_ID' # column by which we group multiple results per molecule
)

The ChEMBLCSV class not only loads the activity data,
but also provides access to it for the
QSAR learning algorithms (see below).

## Model Training

### Exploitation Network

Training the exploitation generator takes a long time (we have over a million molecules in our ZINC sample)
so we would like to monitor
this process. We can use the Monitor
interface for that. The "BasicMonitor" just
saves log files and model checkpoints
in the given directory:

In [ ]:
pr_monitor = BasicMonitor(
        out_dir=OUT_DIR
        , identifier="pr"
    )

TODO: it would be nice to also have a method in
the monitor that would stop the training process

However, we could easily implement our own monitor that could do a bit more than just what the basic monitor does. Here is an example:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

class MyMonitor(BasicMonitor):
    """
    This monitor adds some functionality on top of the basic monitor.
    """
    
    def close(self):
        """
        This method is called after training has completed.
        """
        
        super().close()
        
        # We just get the performance figure.
        return self.getPerfFigure()

In [ ]:
pr_monitor = MyMonitor(
    out_dir=OUT_DIR
    , identifier="pr"
)

The monitor actually does more than just monitoring
of the process. It also keeps track of the best
model built yet and can be used to initialize
a generator based on that.
We use that feature below. If there already is
a network state saved somewhere in our output directory, we do not do any training and just load the model from disk:

In [ ]:
if not pr_monitor.getState(): # this will be False if the monitor cannot find an existing state
    print("Pretraining exploitation network...")
    pretrained = BasicGenerator(
        monitor=pr_monitor
        , corpus=corpus_pre
        , train_params={
            # these parameters are fed directly to the
            # fit method of the underlying pytorch model
            "epochs" : 60 # lets just make this one quick
        }
    )
    pretrained.pretrain()
    # This method also has parameters
    # regarding partioning of the training data.
    # We just use the defaults in this case.
else:
    pretrained = BasicGenerator(
        monitor=pr_monitor
        , initial_state=pr_monitor # the monitor provides initial state
        , corpus=corpus_pre
    )
    # we will not do any training this time,
    # but we could just continue by
    # specifying the training parameters and
    # calling pretrain again
    # TODO: maybe it would be nice if the monitor
    # keeps track of the settings as well

See the figure above? That is from our customized pretrainer monitor. We could configure the monitor to do much more (there are more methods than just the close method that we could override).

### Exploration Network

Next comes the exploration network. The approach is the same, but we use the previously trained network as the initial state. First, we define the monitor, though. We will use the one we defined above, but give it a different identifier:

In [ ]:
ex_monitor = MyMonitor(
        out_dir=OUT_DIR
        , identifier="ex"
    )

The exploration network fine-tunes the pretrained
one so we have to use the pr_monitor to initialize
the initial state of the exploartion network:

In [ ]:
if not ex_monitor.getState():
    print("Pretraining exploration network...")
    exploration = BasicGenerator(
        monitor=ex_monitor
        , initial_state=pr_monitor # initialize from the states of the best pretrained network
        , corpus=corpus_ex # use target-specific corpus for exploration
        , train_params={
            "epochs" : 60 # We will make this one quick too.
        }
    )
    exploration.pretrain(validation_size=512)
    # In this case we want to use a validation set.
    # This set will be used to estimate the
    # loss instead of the training set.
else:
    exploration = BasicGenerator(
        monitor=ex_monitor
        , initial_state=ex_monitor
        , corpus=corpus_ex
    )

### Environment Model

This model will provide the environment for the policy gradient. We already got the data to train this model and saved it to the `environ_data`. This is a data provider for the QSAR model and can be used with other algorithms implemented in the library. However, we will just limit ourselves to random forest in this case:

In [ ]:
# let's see if we can load the model already from disk 
# using the standard deserializer...
identifier = 'environ_rf'
des = FileEnvDeserializer(OUT_DIR, identifier)
try:
    # The deserializer automatically looks for
    # a model in the given directory with the given identifier
    environ_model = des.getModel()
    print("Model found at:", des.path)
except FileNotFoundError:
    # if the model is nowhere to be found, we train and save it
    print("Training environment model...")

    environ_model = RF(train_provider=environ_data)
    environ_model.fit()
    # we save the model so that we don't have to train again next time
    # we also choose to save the performance data (this will
    # also save a ROC curve figure in our output directory
    # to check performance)
    ser = FileEnvSerializer(OUT_DIR, identifier, include_perf=True)
    ser.saveModel(environ_model)

### DrugEx Agent

We now have all ingredients to train
the DrugEx agent. First, we
need to define the policy gradient
strategy:

In [ ]:
policy = PG( # So far this is the only policy there is in the API
    batch_size=512
    , mc=10 # number of repeated samples
    , epsilon=0.01
    , beta=0.1
)

DrugEx agents have their own monitors.
The basic one saves monitoring results to files as well and generally uses the same pattern as we have seen with generators to keep up to date with the best state of the model and so on:

In [ ]:
identifier = 'e_%.2f_%.1f_%dx%d' % (policy.epsilon, policy.beta, policy.batch_size, policy.mc)
agent_monitor = BasicAgentMonitor(OUT_DIR, identifier)

Finally, the DrugEx agent itself:

In [ ]:
if not agent_monitor.getState():
    print("Training DrugEx agent...")
    agent = DrugExAgent(
        agent_monitor # our monitor
        , environ_model # environment for the policy gradient
        , pretrained # the pretrained model
        , policy # our policy gradient implemntation
        , exploration # the fine-tuned model
        , {
            "n_epochs" : 60 # quick again
        }
    )
    agent.train()
else:
    # The DrugEx agent monitor also provides
    # a generator state -> it is the
    # best model from training. We can
    # therefore create a generator 
    # based on this initial state just like we did before:
    agent = BasicGenerator(
        initial_state=agent_monitor
        , corpus=BasicCorpus(
            # If we are not training the generator,
            # we can just provide a basic corpus
            # that only provides vocabulary
            # and no corpus data -> we
            # only have to specify the right
            # vocabulary, which is the one of
            # the exploration or exploitation network.
            # We choose the exploration network here:
            vocabulary=corpus_pre.voc
        )
    )

## Initializing DrugEx Designer

From a fully trained DrugEx agent generator,
we can create a designer class which
will handle sampling of SMILES:

In [ ]:
consumer = CSVConsumer(
    # a CSV file containing not just SMILES, 
    # but also scores as determined by the environment model.
    os.path.join(OUT_DIR, 'designer_mols.csv')
)
designer = BasicDesigner(
    agent=agent # our agent
    , consumer=consumer # use this consumer to return results
    , n_samples=1000 # number of SMILES to sample in total
    , batch_size=512 # number of SMILES to sample in one batch
)
designer() # design the molecules
consumer.save() # save them